# Catch Basin Classifier

Some initial setup code: 

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras import layers
from PIL import Image
from glob import glob
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as pl

## Compute Average Image Size

In [3]:
widths = []
heights = []

for path in glob("data_sorted/**/*.JPG"):
    with Image.open(path) as img:
        widths.append(img.width)
        heights.append(img.height)

In [4]:
image_size = round(sum(widths) / len(widths)), round(sum(heights) / len(heights))
image_size

(554, 732)

## Load and Prepare Data 

Load the data and split into two groups: *training* and *validation*

In [5]:
training_dataset = tf.keras.utils.image_dataset_from_directory("data_sorted", validation_split=0.2, subset="training", seed=321, image_size=image_size)
validation_dataset = tf.keras.utils.image_dataset_from_directory("data_sorted", validation_split=0.2, subset="validation", seed=321, image_size=image_size)

# Get list of classnames to verify that the class names were interpreted correctly
training_dataset.class_names

Found 44 files belonging to 3 classes.
Using 36 files for training.
Found 44 files belonging to 3 classes.
Using 8 files for validation.


['blocked', 'clear', 'partial']

## Construction of the Model

In [45]:
model = Sequential([
    layers.Rescaling(1./255, input_shape=(*image_size, 3)),
    layers.Conv2D(16, 3, padding='same', activation='tanh'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='tanh'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='tanh'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, 554, 732, 3)       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 554, 732, 16)      448       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 277, 366, 16)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 277, 366, 32)      4640      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 138, 183, 32)     0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 138, 183, 64)     

## Training

In [47]:
epochs = 15

history = model.fit(
  training_dataset,
  validation_data=validation_dataset,
  epochs=epochs
)

Epoch 1/15
2/2 [==============================] - 8s 2s/step - loss: 1.2463 - accuracy: 0.6944 - val_loss: 6.0840 - val_accuracy: 0.2500
Epoch 2/15
2/2 [==============================] - 9s 2s/step - loss: 1.0678 - accuracy: 0.6944 - val_loss: 4.6682 - val_accuracy: 0.1250
Epoch 3/15
2/2 [==============================] - 14s 3s/step - loss: 0.4934 - accuracy: 0.8056 - val_loss: 3.0775 - val_accuracy: 0.3750
Epoch 4/15
2/2 [==============================] - 14s 3s/step - loss: 0.2264 - accuracy: 0.9444 - val_loss: 3.7537 - val_accuracy: 0.5000
Epoch 5/15
2/2 [==============================] - 16s 3s/step - loss: 0.6520 - accuracy: 0.7500 - val_loss: 3.2824 - val_accuracy: 0.3750
Epoch 6/15
2/2 [==============================] - 15s 3s/step - loss: 0.5856 - accuracy: 0.7500 - val_loss: 2.6962 - val_accuracy: 0.2500
Epoch 7/15
2/2 [==============================] - 15s 3s/step - loss: 0.4092 - accuracy: 0.7222 - val_loss: 2.8635 - val_accuracy: 0.2500
Epoch 8/15
2/2 [====================

## Training Analysis

In [10]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

print(acc[-1], val_acc[-1])

loss = history.history['loss']
val_loss = history.history['val_loss']

print(loss[-1], val_loss[-1])

0.75 0.5
0.7767490744590759 3.142123222351074


Save the model.

In [9]:
model.save("saved_model")

INFO:tensorflow:Assets written to: saved_model/assets


In [42]:
print(validation_dataset.batch)
for images, labels in validation_dataset.take(1):
    npimages = images.numpy()
    nplabels = labels.numpy()
    results = model.predict(npimages)
    print(results)
    print(nplabels)

<bound method DatasetV2.batch of <BatchDataset shapes: ((None, 554, 732, 3), (None,)), types: (tf.float32, tf.int32)>>
[[-1.9136134   9.529867    1.9984784 ]
 [ 2.449604   10.180234    7.0497146 ]
 [ 0.87548196  9.780648    5.6230073 ]
 [-2.0325027  12.976323   10.247464  ]
 [ 4.391442    6.7047553   4.2867618 ]
 [-3.154632   14.830306    3.8983388 ]
 [ 3.1419604   3.2544844   0.01551011]
 [ 0.13148536 10.271812    0.26384118]]
[1 2 1 1 1 2 0 2]
